# Stock Screener Financial Metrics Test Notebook

This notebook demonstrates how to use the `data_fetcher` module to calculate key financial metrics for stock screening. Metrics include revenue growth, gross margin, P/E, ROE, ROAC, reinvestment rate, and payout ratio (both last year and historical average).

## 1. Import Required Libraries and data_fetcher

Import pandas, numpy, and the data_fetcher module. Set up logging if needed.

In [12]:
import pandas as pd
import numpy as np
import logging
import yfinance as yf

# Import the data_fetcher module
import sys
sys.path.append(r"c:\Users\adamw\Desktop\stock_analysis\dcf_analysis")
from data_fetcher import get_historical_data

# Optional: Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("metrics_test")

In [72]:
def get_share_price(df, ticker):
    share_price = pd.DataFrame(yf.Ticker(ticker).history(period="max", auto_adjust=True, actions=True)["Close"].reset_index())
    share_price.rename(columns={"Date": "date", "Close": "share_price"}, inplace=True)
    # Convert 'date' to date, not datetime
    share_price["date"] = pd.to_datetime(share_price["date"]).dt.date
    # Set Date as index
    share_price.set_index("date", inplace=True)
    # change index from datetime to date
    df.reset_index(inplace=True)
    df["date"] = pd.to_datetime(df["index"]).dt.date
    # Set Date as index
    df.set_index("date", inplace=True)
    # Get the last valid share price for the dates in df
    valid_dates = df.reset_index()["date"].unique()  
    share_prices = []
    for date in valid_dates:
        if date in share_price.index:
            share_prices.append(share_price.loc[date, "share_price"])
        else:
            share_prices.append(share_price[share_price.index <= date]["share_price"].iloc[-1] if not share_price[share_price.index <= date].empty else np.nan)
    return share_prices

## 2. Fetch Financial Data for a Stock

Use `get_historical_data` from data_fetcher to retrieve financial data and valuation info for a given ticker.

In [52]:
# Choose a ticker symbol (e.g., Apple)
# Test Data Fetching for NOVO-B.CO
ticker = 'AAPL'
df, valuation_info = get_historical_data(ticker)
display(df)
print('Valuation Info:', valuation_info)

,FCFF,FCF,EBIT,CapEx,Depreciation And Amortization,Delta WC,Tax Provision,Revenue,EBITDA,Net Income,...,Interest Expense,WorkingCapital,ttm,effective_tax_rate,ReinvestmentRate,capital_invested,ROCE,operating_margin,sales_to_capital,FCFE
2021-09-30,NaN,9.295300e+10,1.118520e+11,1.108500e+10,1.128400e+10,NaN,1.452700e+10,3.658170e+11,1.231360e+11,9.468000e+10,...,2.645000e+09,9.355000e+09,0,0.133023,NaN,1.646720e+11,0.588887,0.305759,2.221489,NaN
2022-09-30,1.284109e+11,1.114430e+11,1.194370e+11,1.070800e+10,1.110400e+10,-2.793200e+10,1.930000e+10,3.943280e+11,1.305410e+11,9.980300e+10,...,2.931000e+09,-1.857700e+10,0,0.162045,-0.061116,1.595060e+11,0.627455,0.302887,2.472183,1.284109e+11
2023-09-30,8.120184e+10,9.958400e+10,1.143010e+11,1.095900e+10,1.151900e+10,1.683500e+10,1.674100e+10,3.832850e+11,1.258200e+11,9.699500e+10,...,3.933000e+09,-1.742000e+09,0,0.147192,0.402962,1.432690e+11,0.680376,0.298214,2.675282,8.120184e+10
2024-09-30,1.171928e+11,1.088070e+11,1.232160e+11,9.447000e+09,1.144500e+10,-2.166300e+10,2.974900e+10,3.910350e+11,1.346610e+11,9.373600e+10,...,NaN,-2.340500e+10,0,0.240912,-0.008249,1.336360e+11,0.699900,0.315102,2.926120,1.171928e+11
2025-08-02,1.284974e+11,9.848600e+10,1.273640e+11,1.107000e+10,1.150200e+10,-3.049100e+10,2.970400e+10,4.003660e+11,1.388660e+11,9.729400e+10,...,NaN,NaN,1,0.233893,-0.081087,1.368200e+11,0.713159,0.318119,2.926224,1.284974e+11


Valuation Info: {'market_cap': 3003400126095.36, 'share_price': 202.38, 'number_shares': 14840399872, 'enterprise_value': 3053029425152, 'total_debt': np.float64(106629000000.0), 'cash': np.float64(29943000000.0), 'net_debt': np.float64(76686000000.0)}


In [74]:
df["share_price"] = get_share_price(df, "AAPL")

In [ ]:
# Define a list of tickers to fetch data for
tickers = ['NOVO-B.CO', 'AAPL', 'MSFT', 'GOOGL']  # Example tickers

# Fetch and combine datasets
dfs = []
valuation_infos = []
for t in tickers:
    df_t, val_info_t = get_historical_data(t)
    df_t = df_t.copy()
    df_t['ticker'] = t
    df_t["share_price"] = get_share_price(df_t, t)
    dfs.append(df_t)
    valuation_infos.append({'ticker': t, **val_info_t})

# Concatenate all dataframes into a single big dataset
big_df = pd.concat(dfs)
big_df = big_df.reset_index().rename(columns={'index': 'date'})

# Optionally, create a DataFrame for valuation info
valuation_df = pd.DataFrame(valuation_infos)

display(big_df)


,date,date,FCFF,FCF,EBIT,CapEx,Depreciation And Amortization,Delta WC,Tax Provision,Revenue,...,ttm,effective_tax_rate,ReinvestmentRate,capital_invested,ROCE,operating_margin,sales_to_capital,FCFE,ticker,share_price
0,2021-12-31,2021-12-31,NaN,4.761500e+10,5.936900e+10,7.385000e+09,6.025000e+09,NaN,1.132300e+10,1.408000e+11,...,0,0.191655,NaN,8.667100e+10,0.553710,0.421655,1.624534,NaN,NOVO-B.CO,347.790863
1,2022-12-31,2022-12-31,4.726291e+10,6.413400e+10,6.944000e+10,1.475300e+10,7.362000e+09,1.175000e+09,1.353700e+10,1.769540e+11,...,0,0.196012,0.416614,9.661700e+10,0.577837,0.392418,1.831500,4.726291e+10,NOVO-B.CO,450.502899
2,2023-12-31,2023-12-31,7.189631e+10,7.001200e+10,1.052160e+11,3.889600e+10,9.413000e+09,-1.726300e+10,2.099100e+10,2.322610e+11,...,0,0.200537,0.368608,1.191750e+11,0.705822,0.453008,1.948907,7.189631e+10,NOVO-B.CO,679.086792
3,2024-12-31,2024-12-31,8.614814e+10,6.965900e+10,1.288310e+11,5.130900e+10,8.545000e+09,-2.662200e+10,2.620300e+10,2.904030e+11,...,0,0.206013,0.323810,2.306180e+11,0.443548,0.443628,1.259238,8.614814e+10,NOVO-B.CO,613.939575
4,2025-08-02,2025-08-02,7.348206e+10,7.435900e+10,1.370550e+11,5.688600e+10,9.461000e+09,-1.253700e+10,2.769100e+10,3.031410e+11,...,1,0.209295,0.492027,2.183250e+11,0.496370,0.452116,1.388485,7.348206e+10,NOVO-B.CO,308.799988
5,2021-09-30,2021-09-30,NaN,9.295300e+10,1.118520e+11,1.108500e+10,1.128400e+10,NaN,1.452700e+10,3.658170e+11,...,0,0.133023,NaN,1.646720e+11,0.588887,0.305759,2.221489,NaN,AAPL,138.682053
6,2022-09-30,2022-09-30,1.284109e+11,1.114430e+11,1.194370e+11,1.070800e+10,1.110400e+10,-2.793200e+10,1.930000e+10,3.943280e+11,...,0,0.162045,-0.061116,1.595060e+11,0.627455,0.302887,2.472183,1.284109e+11,AAPL,136.206757
7,2023-09-30,2023-09-30,8.120184e+10,9.958400e+10,1.143010e+11,1.095900e+10,1.151900e+10,1.683500e+10,1.674100e+10,3.832850e+11,...,0,0.147192,0.402962,1.432690e+11,0.680376,0.298214,2.675282,8.120184e+10,AAPL,169.741714
8,2024-09-30,2024-09-30,1.171928e+11,1.088070e+11,1.232160e+11,9.447000e+09,1.144500e+10,-2.166300e+10,2.974900e+10,3.910350e+11,...,0,0.240912,-0.008249,1.336360e+11,0.699900,0.315102,2.926120,1.171928e+11,AAPL,232.183853
9,2025-08-02,2025-08-02,1.284974e+11,9.848600e+10,1.273640e+11,1.107000e+10,1.150200e+10,-3.049100e+10,2.970400e+10,4.003660e+11,...,1,0.233893,-0.081087,1.368200e+11,0.713159,0.318119,2.926224,1.284974e+11,AAPL,202.380005


,ticker,market_cap,share_price,number_shares,enterprise_value,total_debt,cash,net_debt
0,NOVO-B.CO,1.040193e+12,308.80,3368499968,1448090664960,1.027870e+11,1.565500e+10,8.713200e+10
1,AAPL,3.003400e+12,202.38,14840399872,3053029425152,1.066290e+11,2.994300e+10,7.668600e+10
2,MSFT,3.895799e+12,524.11,7433169920,3913697722368,6.058800e+10,3.024200e+10,3.034600e+10
3,GOOGL,1.100169e+12,189.13,5816999936,2233858260992,2.546100e+10,2.346600e+10,1.995000e+09


In [90]:
def get_cagr(series):
    """Calculate CAGR given a pandas Series (oldest to newest)."""
    series = series.dropna()
    if len(series) < 2:
        return np.nan
    start, end = series.iloc[0], series.iloc[-1]
    n = len(series) - 1
    if start <= 0 or end <= 0:
        return np.nan
    return (end / start) ** (1 / n) - 1

df = big_df.copy()
df["EPS"] = df["Net Income"] / df["Ordinary Shares Number"]
df["EPS Growth Rate"] = df.groupby("ticker")["EPS"].pct_change() * 100
df["PE Ratio"] = df["share_price"] / df["EPS"]
df["PEG Ratio"] = df["PE Ratio"] / df["EPS Growth Rate"]
df["book_value"] = df["Total Assets"] - df["Total Debt"]
df["book_value_per_share"] = df["book_value"] / df["Ordinary Shares Number"]
df["PB Ratio"] = df["share_price"] / df["book_value_per_share"]
df["graham_number"] = np.sqrt(22.5 * df["EPS"] * df["book_value_per_share"])
df["p_to_graham"] = df["share_price"] / df["graham_number"]
df["dividends_per_share"] = df["Dividends"] / df["Ordinary Shares Number"]
df["dividend_yield"] = df["dividends_per_share"] / df["share_price"]
df["dividend_payout_ratio"] = df["dividends_per_share"] / df["EPS"]
df["debt_to_equity"] = df["Total Debt"] / df["book_value"]
df["current_ratio"] = df["Current Assets"] / df["Current Liabilities"]
#df["quick_ratio"] = (df["Current Assets"] - df["Inventory"]) / df["Current Liabilities"]
df["interest_coverage"] = df["EBIT"] / df["Interest Expense"]
df["fcf_per_share"] = df["FCF"] / df["Ordinary Shares Number"]
df["fcf_yield"] = df["fcf_per_share"] / df["share_price"]


In [ ]:


df['revenue_growth_pct'] = df.groupby('ticker')['Revenue'].apply(lambda x: x.pct_change() * 100)

TypeError: incompatible index of inserted column with frame index

In [93]:

df['ebit_growth_pct'] = df.groupby('ticker')['EBIT'].apply(lambda x: x.pct_change() * 100)


TypeError: incompatible index of inserted column with frame index

In [ ]:

df['net_income_growth_pct'] = df.groupby('ticker')['Net Income'].apply(lambda x: x.pct_change() * 100)
df['fcf_growth_pct'] = df.groupby('ticker')['FCF'].apply(lambda x: x.pct_change() * 100)
df['roic_trend_pct'] = df.groupby('ticker')['ROIC'].apply(lambda x: x.pct_change() * 100)

"""df['cagr_revenue'] = df.groupby('ticker')['Revenue'].apply(get_cagr)
df['cagr_ebit'] = df.groupby('ticker')['EBIT'].apply(get_cagr)
df['cagr_net_income'] = df.groupby('ticker')['Net Income'].apply(get_cagr)
df['cagr_fcf'] = df.groupby('ticker')['FCF'].apply(get_cagr)
df['cagr_eps'] = df.groupby('ticker')['EPS'].apply(get_cagr)"""

In [87]:
df

,date,date,FCFF,FCF,EBIT,CapEx,Depreciation And Amortization,Delta WC,Tax Provision,Revenue,...,EPS Growth Rate,PEG Ratio,book_value,book_value_per_share,PB Ratio,graham_number,p_to_graham,dividends_per_share,dividend_yield,dividend_payout_ratio
0,2021-12-31,2021-12-31,NaN,4.761500e+10,5.936900e+10,7.385000e+09,6.025000e+09,NaN,1.132300e+10,1.408000e+11,...,NaN,NaN,1.678630e+11,36.333983,9.572055,91.927467,3.783318,NaN,NaN,NaN
1,2022-12-31,2022-12-31,4.726291e+10,6.413400e+10,6.944000e+10,1.475300e+10,7.362000e+09,1.175000e+09,1.353700e+10,1.769540e+11,...,19.371402,1.884695,2.154730e+11,47.885017,9.408014,115.302510,3.907139,NaN,NaN,NaN
2,2023-12-31,2023-12-31,7.189631e+10,7.001200e+10,1.052160e+11,3.889600e+10,9.413000e+09,-1.726300e+10,2.099100e+10,2.322610e+11,...,52.115606,0.694206,2.874800e+11,64.482152,10.531392,165.023420,4.115093,NaN,NaN,NaN
3,2024-12-31,2024-12-31,8.614814e+10,6.965900e+10,1.288310e+11,5.130900e+10,8.545000e+09,-2.662200e+10,2.620300e+10,2.904030e+11,...,21.139723,1.277234,3.630080e+11,81.733883,7.511445,204.489106,3.002309,NaN,NaN,NaN
4,2025-08-02,2025-08-02,7.348206e+10,7.435900e+10,1.370550e+11,5.688600e+10,9.461000e+09,-1.253700e+10,2.769100e+10,3.031410e+11,...,3.591516,3.650225,3.704390e+11,83.407026,3.702326,210.248315,1.468739,NaN,NaN,NaN
5,2021-09-30,2021-09-30,NaN,9.295300e+10,1.118520e+11,1.108500e+10,1.128400e+10,NaN,1.452700e+10,3.658170e+11,...,NaN,NaN,2.144800e+11,13.056723,10.621505,41.149180,3.370226,0.880696,0.006350,0.152799
6,2022-09-30,2022-09-30,1.284109e+11,1.114430e+11,1.194370e+11,1.070800e+10,1.110400e+10,-2.793200e+10,1.930000e+10,3.943280e+11,...,8.606626,2.528155,2.202750e+11,13.816040,9.858596,44.112740,3.087697,0.930854,0.006834,0.148703
7,2023-09-30,2023-09-30,8.120184e+10,9.958400e+10,1.143010e+11,1.095900e+10,1.151900e+10,1.683500e+10,1.674100e+10,3.832850e+11,...,-0.355054,-76.643846,2.414950e+11,15.530164,10.929808,46.686133,3.635806,0.966234,0.005692,0.154905
8,2024-09-30,2024-09-30,1.171928e+11,1.088070e+11,1.232160e+11,9.447000e+09,1.144500e+10,-2.166300e+10,2.974900e+10,3.910350e+11,...,-0.590085,-63.455654,2.583510e+11,17.090339,13.585678,48.830377,4.754906,1.007754,0.004340,0.162520
9,2025-08-02,2025-08-02,1.284974e+11,9.848600e+10,1.273640e+11,1.107000e+10,1.150200e+10,-3.049100e+10,2.970400e+10,4.003660e+11,...,5.028805,6.179417,2.330470e+11,15.599577,12.973429,47.810729,4.232941,1.025014,0.005065,0.157389


In [7]:
# Add Growth Metrics columns to valuation_df

def get_cagr(series):
    """Calculate CAGR given a pandas Series (oldest to newest)."""
    series = series.dropna()
    if len(series) < 2:
        return np.nan
    start, end = series.iloc[0], series.iloc[-1]
    n = len(series) - 1
    if start <= 0 or end <= 0:
        return np.nan
    return (end / start) ** (1 / n) - 1

def get_eps_growth(df_t, shares):
    """Calculate EPS CAGR for a ticker's dataframe and number of shares."""
    eps = df_t['Net Income'] / shares
    return get_cagr(eps)

def get_fcf_growth(df_t):
    """Calculate FCF CAGR for a ticker's dataframe."""
    return get_cagr(df_t['FCF'])

def get_roic_trend(df_t):
    """Calculate ROCE trend (CAGR) for a ticker's dataframe."""
    return get_cagr(df_t['ROCE'])

def get_revenue_growth(df_t):
    """Calculate Revenue CAGR for a ticker's dataframe."""
    return get_cagr(df_t['Revenue'])

# Add columns to valuation_df
valuation_df['revenue_growth_pct'] = np.nan
valuation_df['eps_growth_pct'] = np.nan
valuation_df['fcf_growth_pct'] = np.nan
valuation_df['roic_trend_pct'] = np.nan

for idx, row in valuation_df.iterrows():
    ticker = row['ticker']
    shares = row['number_shares']
    df_t = big_df[big_df['ticker'] == ticker].sort_values('date')
    valuation_df.at[idx, 'revenue_growth_pct'] = get_revenue_growth(df_t) * 100
    valuation_df.at[idx, 'eps_growth_pct'] = get_eps_growth(df_t, shares) * 100
    valuation_df.at[idx, 'fcf_growth_pct'] = get_fcf_growth(df_t) * 100
    valuation_df.at[idx, 'roic_trend_pct'] = get_roic_trend(df_t) * 100

# Display with new columns
display(valuation_df[['ticker', 'revenue_growth_pct', 'eps_growth_pct', 'fcf_growth_pct', 'roic_trend_pct']])

,ticker,revenue_growth_pct,eps_growth_pct,fcf_growth_pct,roic_trend_pct
0,NOVO-B.CO,21.132451,21.657596,11.788580,-2.695967
1,AAPL,2.281792,0.683187,1.456008,4.903174
2,MSFT,9.179653,8.775410,2.392464,-5.520364
3,GOOGL,9.574153,11.035938,-0.106120,1.044166


In [8]:
# Add profitability metrics to valuation_df

def get_last_row(ticker):
    df_t = big_df[big_df['ticker'] == ticker]
    last = df_t[df_t['ttm'] == 0].sort_values('date').iloc[-1]
    return last

def roe(row):
    last = get_last_row(row['ticker'])
    equity = last['Total Assets'] - last['Total Debt']
    return last['Net Income'] / equity if equity else np.nan

def roa(row):
    last = get_last_row(row['ticker'])
    return last['Net Income'] / last['Total Assets'] if last['Total Assets'] else np.nan

def roic(row):
    last = get_last_row(row['ticker'])
    # NOPAT = EBIT * (1 - effective_tax_rate)
    nopat = last['EBIT'] * (1 - last['effective_tax_rate'])
    invested_capital = last['capital_invested']
    return nopat / invested_capital if invested_capital else np.nan

def gross_margin(row):
    last = get_last_row(row['ticker'])
    # Gross Margin = (Revenue - Cost of Goods Sold) / Revenue
    # If COGS not available, use EBITDA as proxy for gross profit
    if 'COGS' in last and pd.notna(last['COGS']):
        gross_profit = last['Revenue'] - last['COGS']
    else:
        gross_profit = last['EBITDA']
    return gross_profit / last['Revenue'] if last['Revenue'] else np.nan

def operating_margin(row):
    last = get_last_row(row['ticker'])
    return last['operating_margin']

def net_margin(row):
    last = get_last_row(row['ticker'])
    return last['Net Income'] / last['Revenue'] if last['Revenue'] else np.nan

valuation_df['roe'] = valuation_df.apply(roe, axis=1)
valuation_df['roa'] = valuation_df.apply(roa, axis=1)
valuation_df['roic'] = valuation_df.apply(roic, axis=1)
valuation_df['gross_margin'] = valuation_df.apply(gross_margin, axis=1)
valuation_df['operating_margin'] = valuation_df.apply(operating_margin, axis=1)
valuation_df['net_margin'] = valuation_df.apply(net_margin, axis=1)

display(valuation_df[['ticker', 'roe', 'roa', 'roic', 'gross_margin', 'operating_margin', 'net_margin']])

,ticker,roe,roa,roic,gross_margin,operating_margin,net_margin
0,NOVO-B.CO,0.278198,0.216808,0.443548,0.473053,0.443628,0.347751
1,AAPL,0.362824,0.256825,0.699900,0.344371,0.315102,0.239713
2,MSFT,0.182359,0.164510,0.277661,0.568517,0.447289,0.361460
3,GOOGL,0.235685,0.222358,0.306782,0.386820,0.343077,0.286037


In [11]:
# Add TTM Income/Dividend metrics to valuation_df

def dividend_yield_ttm(row):
    ticker = row['ticker']
    df_t = big_df[big_df['ticker'] == ticker]
    last = df_t.sort_values('date').iloc[-1]
    shares = last['Ordinary Shares Number'] if 'Ordinary Shares Number' in last else row.get('number_shares', np.nan)
    dividend_per_share = last['Dividends'] / shares if shares else np.nan
    price = row['share_price']
    return dividend_per_share / price if pd.notna(dividend_per_share) and pd.notna(price) and price != 0 else np.nan

def payout_ratio_ttm(row):
    ticker = row['ticker']
    df_t = big_df[big_df['ticker'] == ticker]
    last = df_t.sort_values('date').iloc[-1]
    annual_div = abs(last['Dividends']) if pd.notna(last['Dividends']) else np.nan
    net_income = last['Net Income']
    return annual_div / net_income if pd.notna(annual_div) and pd.notna(net_income) and net_income != 0 else np.nan

def fcf_payout_ratio_ttm(row):
    ticker = row['ticker']
    df_t = big_df[big_df['ticker'] == ticker]
    last = df_t.sort_values('date').iloc[-1]
    annual_div = abs(last['Dividends']) if pd.notna(last['Dividends']) else np.nan
    fcf = last['FCF']
    return annual_div / fcf if pd.notna(annual_div) and pd.notna(fcf) and fcf != 0 else np.nan

valuation_df['dividend_yield_ttm'] = valuation_df.apply(dividend_yield_ttm, axis=1)
valuation_df['payout_ratio_ttm'] = valuation_df.apply(payout_ratio_ttm, axis=1)
valuation_df['fcf_payout_ratio_ttm'] = valuation_df.apply(fcf_payout_ratio_ttm, axis=1)

display(valuation_df[['ticker', 'dividend_yield', 'dividend_yield_ttm', 'payout_ratio', 'payout_ratio_ttm', 'fcf_payout_ratio', 'fcf_payout_ratio_ttm', 'dividend_growth_rate']])


KeyError: "['dividend_yield', 'payout_ratio', 'fcf_payout_ratio', 'dividend_growth_rate'] not in index"

In [10]:
big_df

,date,FCFF,FCF,EBIT,CapEx,Depreciation And Amortization,Delta WC,Tax Provision,Revenue,EBITDA,...,WorkingCapital,ttm,effective_tax_rate,ReinvestmentRate,capital_invested,ROCE,operating_margin,sales_to_capital,FCFE,ticker
0,2021-12-31,NaN,4.761500e+10,5.936900e+10,7.385000e+09,6.025000e+09,NaN,1.132300e+10,1.408000e+11,6.539400e+10,...,-1.392100e+10,0,0.191655,NaN,8.667100e+10,0.553710,0.421655,1.624534,NaN,NOVO-B.CO
1,2022-12-31,4.726291e+10,6.413400e+10,6.944000e+10,1.475300e+10,7.362000e+09,1.175000e+09,1.353700e+10,1.769540e+11,7.680200e+10,...,-1.274600e+10,0,0.196012,0.416614,9.661700e+10,0.577837,0.392418,1.831500,4.726291e+10,NOVO-B.CO
2,2023-12-31,7.189631e+10,7.001200e+10,1.052160e+11,3.889600e+10,9.413000e+09,-1.726300e+10,2.099100e+10,2.322610e+11,1.146290e+11,...,-3.000900e+10,0,0.200537,0.368608,1.191750e+11,0.705822,0.453008,1.948907,7.189631e+10,NOVO-B.CO
3,2024-12-31,8.614814e+10,6.965900e+10,1.288310e+11,5.130900e+10,8.545000e+09,-2.662200e+10,2.620300e+10,2.904030e+11,1.373760e+11,...,-5.663100e+10,0,0.206013,0.323810,2.306180e+11,0.443548,0.443628,1.259238,8.614814e+10,NOVO-B.CO
4,2025-08-02,7.348206e+10,7.435900e+10,1.370550e+11,5.688600e+10,9.461000e+09,-1.253700e+10,2.769100e+10,3.031410e+11,1.465160e+11,...,NaN,1,0.209295,0.492027,2.183250e+11,0.496370,0.452116,1.388485,7.348206e+10,NOVO-B.CO
5,2021-09-30,NaN,9.295300e+10,1.118520e+11,1.108500e+10,1.128400e+10,NaN,1.452700e+10,3.658170e+11,1.231360e+11,...,9.355000e+09,0,0.133023,NaN,1.646720e+11,0.588887,0.305759,2.221489,NaN,AAPL
6,2022-09-30,1.284109e+11,1.114430e+11,1.194370e+11,1.070800e+10,1.110400e+10,-2.793200e+10,1.930000e+10,3.943280e+11,1.305410e+11,...,-1.857700e+10,0,0.162045,-0.061116,1.595060e+11,0.627455,0.302887,2.472183,1.284109e+11,AAPL
7,2023-09-30,8.120184e+10,9.958400e+10,1.143010e+11,1.095900e+10,1.151900e+10,1.683500e+10,1.674100e+10,3.832850e+11,1.258200e+11,...,-1.742000e+09,0,0.147192,0.402962,1.432690e+11,0.680376,0.298214,2.675282,8.120184e+10,AAPL
8,2024-09-30,1.171928e+11,1.088070e+11,1.232160e+11,9.447000e+09,1.144500e+10,-2.166300e+10,2.974900e+10,3.910350e+11,1.346610e+11,...,-2.340500e+10,0,0.240912,-0.008249,1.336360e+11,0.699900,0.315102,2.926120,1.171928e+11,AAPL
9,2025-08-02,1.284974e+11,9.848600e+10,1.273640e+11,1.107000e+10,1.150200e+10,-3.049100e+10,2.970400e+10,4.003660e+11,1.388660e+11,...,NaN,1,0.233893,-0.081087,1.368200e+11,0.713159,0.318119,2.926224,1.284974e+11,AAPL
